In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

pd.set_option('display.float_format', '{:,.4f}'.format)

# VAR FX

In [1]:
def var_es(data, positions, nc, long):
    rend_inversion = ((data * positions).sum(axis=1)).pct_change().dropna()
    if long == True:
        percentil =  (1-nc)*100
        var_p = np.percentile(rend_inversion, percentil)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = rend_inversion[rend_inversion < var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    else:
        percentil = nc*100
        var_p = np.percentile(rend_inversion, percentil)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = rend_inversion[rend_inversion > var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    resultados = pd.DataFrame({'VAR %':var_p, 'VAR $':var_m, 'ES %':es_p, 'ES $':es_m}, index=['Resultados'])
    return resultados

# VAR EQUITIES

In [4]:
def var_stocks(data, positions, nc, long):
    rend = data.pct_change().dropna()
    port_value = (data.iloc[-1] * positions).sum()
    w = (data*positions).iloc[-1] / port_value
    port_rend = np.dot(w, rend.T)
    
    if long == True:
        var_p = np.percentile(port_rend, 100-nc)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = port_rend[port_rend < var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    else:
        var_p = np.percentile(port_rend, nc)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = port_rend[port_rend > var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
        
    resultados = pd.DataFrame({
        '':['%', '$'],
        'VaR':[var_p, var_m],
        'C-VaR':[es_p, es_m]
        })
    return resultados

# DELTA HEDGE

In [3]:
call_blsdelta = lambda s, k, r, sigma, T: norm.cdf( ( np.log(s/k) + (r + sigma**2 / 2) * T ) / ( sigma * np.sqrt(T)) )
put_blsdelta = lambda s, k, r, sigma, T: np.abs(norm.cdf( ( np.log(s/k) + (r + sigma**2 / 2) * T ) / ( sigma * np.sqrt(T)) ) - 1)

def delta_hedge(calls, puts, noc_c, noc_p):
    return np.array([call_blsdelta(*i) for i in calls]).dot(noc_c) - np.array([put_blsdelta(*i) for i in puts]).dot(noc_p)